In [10]:
from pathlib import Path
from datetime import datetime, timedelta, date
import gzip
import json
import pandas as pd
import re
from tqdm import tqdm

In [25]:
RAW_DIR = Path("bmrs_json_raw")
ACCEPTANCES_INPUT_DIR = RAW_DIR / "BIDOFFER_ACCEPTANCES"
PRICES_INPUT_DIR = RAW_DIR / "BIDOFFER_PRICES"

PROCESSED_DIR = Path("bmrs_csv_filled")
OUTPUT_ACCEPTANCES_DIR = PROCESSED_DIR / "BIDOFFER_ACCEPTANCES.csv"
OUTPUT_PRICES_DIR = PROCESSED_DIR / "BIDOFFER_PRICES.csv"

def load_boalf(input_acceptances_dir: Path, input_prices_dir: Path) -> pd.DataFrame:
    """
    Loads and flattens all .json.gz acceptance files and merges with bid/offer prices
    using acceptanceNumber, acceptanceTime, and bmUnit.
    Returns a DataFrame with one row per BOALF entry including bidPrice and offerPrice.
    """
    # --- Load acceptances (volumes, timings, flags) ---
    accept_rows = []
    for file in tqdm(sorted(input_acceptances_dir.glob("*.json.gz")), desc="Loading BOALF files"):
        try:
            with gzip.open(file, 'rt', encoding='utf-8') as f:
                data = json.load(f)
            entries = data['data'] if isinstance(data, dict) and 'data' in data else data
            for entry in entries:
                accept_rows.append({
                    'settlementPeriodFrom': entry.get('settlementPeriodFrom'),
                    'settlementPeriodTo': entry.get('settlementPeriodTo'),
                    'levelFrom': float(entry.get('levelFrom', 0)),
                    'levelTo': float(entry.get('levelTo', 0)),
                    'timeFrom': entry.get('timeFrom'),
                    'timeTo': entry.get('timeTo'),
                    'bmUnit': entry.get('bmUnit'),
                    'nationalGridBmUnit': entry.get('nationalGridBmUnit'),
                    'acceptanceNumber': entry.get('acceptanceNumber'),
                    'acceptanceTime': entry.get('acceptanceTime'),
                    'soFlag': entry.get('soFlag', False),
                    'storFlag': entry.get('storFlag', False),
                })
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    accept_df = pd.DataFrame(accept_rows)
    accept_df['timeFrom'] = pd.to_datetime(accept_df['timeFrom']).dt.tz_localize(None)
    accept_df['timeTo'] = pd.to_datetime(accept_df['timeTo']).dt.tz_localize(None)
    accept_df['acceptanceTime'] = pd.to_datetime(accept_df['acceptanceTime']).dt.tz_localize(None)

    # --- Load prices (bidPrice, offerPrice) ---
    price_rows = []
    for file in tqdm(sorted(input_prices_dir.glob("*.json.gz")), desc="Loading price files"):
        try:
            with gzip.open(file, 'rt', encoding='utf-8') as f:
                data = json.load(f)
            entries = data['data'] if isinstance(data, dict) and 'data' in data else data
            for entry in entries:
                price_rows.append({
                    'bmUnit': entry.get('bmUnit'),
                    'acceptanceNumber': entry.get('acceptanceNumber'),
                    'acceptanceTime': entry.get('acceptanceTime'),
                    'bidPrice': entry.get('bidPrice'),
                    'offerPrice': entry.get('offerPrice'),
                })
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    price_df = pd.DataFrame(price_rows)
    price_df['acceptanceTime'] = pd.to_datetime(price_df['acceptanceTime']).dt.tz_localize(None)

    # --- Merge on composite key ---
    merged_df = pd.merge(
        accept_df,
        price_df,
        on=['bmUnit', 'acceptanceNumber', 'acceptanceTime'],
        how='left'
    )

    # Sort by acceptanceTime
    merged_df = merged_df.sort_values('acceptanceTime').reset_index(drop=True)

    return merged_df

In [26]:
df_boalf = load_boalf(input_acceptances_dir=ACCEPTANCES_INPUT_DIR, input_prices_dir=PRICES_INPUT_DIR)
df_boalf.head()

Loading price files: 100%|██████████| 70072/70072 [00:19<00:00, 3645.37it/s]


,settlementPeriodFrom,settlementPeriodTo,levelFrom,levelTo,timeFrom,timeTo,bmUnit,nationalGridBmUnit,acceptanceNumber,acceptanceTime,soFlag,storFlag,bidPrice,offerPrice
0,1,1,0.0,0.0,2021-06-29 23:01:00,2021-06-29 23:02:00,T_DINO-6,DINO-6,131996,2021-06-29 22:00:00,False,False,NaN,NaN
1,1,1,130.0,100.0,2021-06-29 23:21:00,2021-06-29 23:23:00,T_CARR-2,CARR-2,67550,2021-06-29 22:01:00,True,False,49.0,125.0
2,1,1,176.0,130.0,2021-06-29 23:19:00,2021-06-29 23:21:00,T_CARR-2,CARR-2,67550,2021-06-29 22:01:00,True,False,49.0,125.0
3,1,1,100.0,0.0,2021-06-29 23:23:00,2021-06-29 23:29:00,T_CARR-2,CARR-2,67550,2021-06-29 22:01:00,True,False,49.0,125.0
4,1,1,190.0,0.0,2021-06-29 23:05:00,2021-06-29 23:29:00,T_CDCL-1,CDCL-1,118989,2021-06-29 22:02:00,False,False,16.6,98.0


In [18]:
# Load BMU data
BMU_DIR = Path("other")
ORIGINAL_BMU_PATH = BMU_DIR / "BMU_Dataset.csv"
OUTPUT_BMU_PATH = BMU_DIR / "BMU_Dataset_Grouped.csv"
df_bmu = pd.read_csv(ORIGINAL_BMU_PATH)

# Define modular grouping dictionary
# FUEL_GROUP_MAPPING = {
#     'THERMAL_GAS': ['GAS', 'CCGT', 'OCGT'],
#     'THERMAL_OTHER': ['COAL', 'DIESEL', 'BIOMASS'],
#     'STORAGE_BATTERY': ['BATTERY'],
#     'STORAGE_PSH': ['PS', 'PS '],
#     'RENEWABLE_WIND': ['WIND'],
#     'RENEWABLE_SOLAR': ['SOLAR'],
#     'RENEWABLE_HYDRO': ['NPSHYD', 'TIDAL'],
#     'INTERCONNECTOR': ['INTBN', 'INTIRL', 'INTEW', 'INTFR', 'INTNEM', 'INTGRNL', 'INTGBR'],
#     'LOAD_RESPONSE': ['LOAD RESPONSE'],
#     'NUCLEAR': ['NUCLEAR']
#     # 'OTHER' will be handled as fallback
# }

FUEL_GROUP_MAPPING = {
    'GAS': ['GAS', 'CCGT', 'OCGT'],
    # 'OTHER' will be handled as fallback
}

# Flatten the dictionary into a reverse lookup
fuel_to_group = {
    fuel_type: group
    for group, fuel_list in FUEL_GROUP_MAPPING.items()
    for fuel_type in fuel_list
}

# Apply grouping
df_bmu["FUEL_TYPE_CLEAN"] = df_bmu["REG_FUEL_TYPE"].map(fuel_to_group)
df_bmu["FUEL_TYPE_CLEAN"] = df_bmu["FUEL_TYPE_CLEAN"].fillna("OTHER")

# Save updated dataframe
df_bmu.to_csv(OUTPUT_BMU_PATH, index=False)
print("✓ FUEL_TYPE_CLEAN column added and saved.")


✓ FUEL_TYPE_CLEAN column added and saved.


In [30]:
before = len(df_boalf)
df_boalf = df_boalf[~((df_boalf["levelFrom"] == 0) & (df_boalf["levelTo"] == 0))]
after = len(df_boalf)
print(f"✓ Removed {before - after} zero-volume entries")

before = len(df_boalf)
df_boalf = df_boalf.drop_duplicates()
after = len(df_boalf)
print(f"✓ Removed {before - after} duplicate rows")

✓ Removed 0 zero-volume entries
✓ Removed 4302848 duplicate rows


In [27]:
# Load cleaned BMU fuel type mapping
df_bmu = pd.read_csv(BMU_DIR / "BMU_Dataset_Grouped.csv")

# Build a mapping from nationalGridBmUnit to FUEL_TYPE_CLEAN
bmu_fuel_map = df_bmu.set_index("nationalGridBmUnit")["FUEL_TYPE_CLEAN"].to_dict()

# Map onto df_boalf
df_boalf["fuelType"] = df_boalf["nationalGridBmUnit"].map(bmu_fuel_map)

missing = df_boalf["fuelType"].isna().sum()
print(f"✓ fuelType mapping complete. {missing} unmatched BMUs.")

# Fill missing fuelType values with 'OTHER'
df_boalf["fuelType"] = df_boalf["fuelType"].fillna("OTHER")
print(f"✓ Missing fuelType values filled with 'OTHER'. Total rows: {len(df_boalf)}")

# Save the processed DataFrame
df_boalf.to_parquet("df_boalf.parquet", index=False)
print("✓ Processed DataFrame saved to df_boalf.parquet.")

✓ fuelType mapping complete. 222543 unmatched BMUs.
✓ Missing fuelType values filled with 'OTHER'. Total rows: 16419562
✓ Processed DataFrame saved to df_boalf.parquet.


In [28]:
df_boalf = pd.read_parquet("df_boalf.parquet")

# Convert timeFrom and timeTo to Naive
df_boalf['timeFrom'] = pd.to_datetime(df_boalf['timeFrom']).dt.tz_localize(None)
df_boalf['timeTo'] = pd.to_datetime(df_boalf['timeTo']).dt.tz_localize(None)

print("✓ timeFrom and timeTo columns converted")

✓ timeFrom and timeTo columns converted


In [21]:
from datetime import datetime, timedelta, date
from zoneinfo import ZoneInfo
import pandas as pd
from tqdm import tqdm
import numpy as np

def build_uk_halfhour_calendar(start_date, end_date):
    """
    Build UK half-hour calendar with correct DST handling:
      • Spring-forward days: 46 periods (including the skipped 01:00/01:30)
      • Normal days: 48 periods 00:00-23:30
      • BST days: 48 periods 23:00(prev day)-22:30
      • Autumn-back days: 50 periods 23:00(prev day)-22:30
    """

    def _to_date(x):
        if isinstance(x, str):
            
            if x.count("-") == 2 and x[4] == "-": # ISO format
                return date.fromisoformat(x)
            return datetime.strptime(x, "%d/%m/%Y").date() # UK format
        if isinstance(x, pd.Timestamp):
            return x.date()
        return x

    start = _to_date(start_date)
    end   = _to_date(end_date)

    london = ZoneInfo("Europe/London")
    utc    = ZoneInfo("UTC")
    rows   = []

    for single in pd.date_range(start, end, freq="D"):
        D   = single.date()
        # local midnights in London
        dt0 = datetime(D.year, D.month, D.day, tzinfo=london)
        dt1 = dt0 + timedelta(days=1)

        # number of half-hours that actually occur
        total_secs = (dt1.astimezone(utc) - dt0.astimezone(utc)).total_seconds()
        n_periods = int(total_secs // 1800)

        # align to UTC-naive base for SP1
        offset_h = dt0.utcoffset().total_seconds() / 3600
        if offset_h > 0:
            base = datetime(D.year, D.month, D.day) - timedelta(hours=int(offset_h))
        else:
            base = datetime(D.year, D.month, D.day)

        for i in range(n_periods):
            rows.append({
                "startTime":        base + timedelta(minutes=30 * i),
                "settlementDate":   D,
                "settlementPeriod": i + 1
            })

    df = pd.DataFrame(rows)

    # ─── coerce to pandas time types ───
    df["startTime"]      = pd.to_datetime(df["startTime"])
    df["settlementDate"] = pd.to_datetime(df["settlementDate"]).dt.normalize()
    df["settlementPeriod"] = df["settlementPeriod"].astype("int32")
    # ───────────────────────────────────

    return df

calendar = build_uk_halfhour_calendar(df_boalf["timeFrom"].min(), df_boalf["timeTo"].max())

In [22]:
def process_batches_interpolated(df_boalf, calendar, days_per_batch=1):
    """
    Process BOALF acceptances into MWh per settlement period using exact trapezium
    integration with linear interpolation at settlement period boundaries.
    """
    all_results = []

    start = calendar["settlementDate"].min().date()
    end = calendar["settlementDate"].max().date()
    batch_dates = pd.date_range(start, end, freq=f"{days_per_batch}D")

    for i in tqdm(range(len(batch_dates)), desc="Processing batches"):
        batch_start = batch_dates[i].date()
        batch_end = (batch_start + timedelta(days=days_per_batch - 1))
        batch_mask = (calendar["settlementDate"].dt.date >= batch_start) & \
                     (calendar["settlementDate"].dt.date <= batch_end)
        calendar_batch = calendar.loc[batch_mask].copy()

        if calendar_batch.empty:
            continue

        cal_start = calendar_batch["startTime"].min()
        cal_end = calendar_batch["startTime"].max() + timedelta(minutes=30)

        # Subset BOALF data overlapping this batch
        df_subset = df_boalf[(df_boalf["timeTo"] > cal_start) &
                             (df_boalf["timeFrom"] < cal_end)].copy()
        if df_subset.empty:
            continue

        # Precompute acceptance durations for interpolation scaling
        df_subset["duration_total"] = (df_subset["timeTo"] - df_subset["timeFrom"]).dt.total_seconds()
        df_subset["levelFrom"] = df_subset["levelFrom"].astype(float)
        df_subset["levelTo"] = df_subset["levelTo"].astype(float)

        rows = []
        for _, sp in calendar_batch.iterrows():
            sp_start = sp["startTime"]
            sp_end = sp_start + timedelta(minutes=30)

            # Find overlapping acceptances
            mask = (df_subset["timeTo"] > sp_start) & (df_subset["timeFrom"] < sp_end)
            overlapping = df_subset.loc[mask].copy()
            if overlapping.empty:
                continue

            # Clamp overlap start/end to SP boundaries
            overlap_start = np.maximum(overlapping["timeFrom"].values.astype("datetime64[ns]"),
                                       np.datetime64(sp_start))
            overlap_end = np.minimum(overlapping["timeTo"].values.astype("datetime64[ns]"),
                                     np.datetime64(sp_end))
            overlap_secs = (overlap_end - overlap_start) / np.timedelta64(1, "s")

            # Calculate position within acceptance for interpolation
            frac_start = (overlap_start - overlapping["timeFrom"].values.astype("datetime64[ns]")) \
                         / np.timedelta64(1, "s") / overlapping["duration_total"].values
            frac_end = (overlap_end - overlapping["timeFrom"].values.astype("datetime64[ns]")) \
                       / np.timedelta64(1, "s") / overlapping["duration_total"].values

            # Interpolated MW at start and end of overlap
            power_start = overlapping["levelFrom"].values + \
                          (overlapping["levelTo"].values - overlapping["levelFrom"].values) * frac_start
            power_end = overlapping["levelFrom"].values + \
                        (overlapping["levelTo"].values - overlapping["levelFrom"].values) * frac_end

            # Trapezium integration for this SP slice
            mwh = ((power_start + power_end) / 2) * (overlap_secs / 3600.0)

            result = pd.DataFrame({
                "startTime": sp["startTime"],
                "settlementDate": sp["settlementDate"],
                "settlementPeriod": sp["settlementPeriod"],
                "fuelType": overlapping["fuelType"].values,
                "MWh": mwh,
                "acceptanceNumber": overlapping["acceptanceNumber"].values,
                "soFlag": overlapping["soFlag"].values,
                "storFlag": overlapping["storFlag"].values,
            })

            rows.append(result)

        if rows:
            all_results.append(pd.concat(rows))

    return pd.concat(all_results)

df_energy = process_batches_interpolated(df_boalf, calendar, days_per_batch=1)

Processing batches: 100%|██████████| 1464/1464 [01:26<00:00, 16.88it/s]


In [23]:
FINAL_OUTPUT_DIR = Path("bmrs_csv_filled")

def make_wide_variants(df_energy, name_prefix, calendar):
    """
    Generates absolute and percentage CSVs from df_energy.
    Produces 2 files: ABS and PCT.
    Ensures output matches the full calendar length and shows empty SPs.
    """
    # Group and pivot to wide format (absolute MWh)
    df_abs_raw = (
        df_energy
        .groupby(["startTime", "settlementDate", "settlementPeriod", "fuelType"], as_index=False)
        .agg({"MWh": "sum"})
        .pivot(index=["startTime", "settlementDate", "settlementPeriod"], columns="fuelType", values="MWh")
        .reset_index()
    )

    # Merge with full calendar to ensure every SP is included
    df_abs = calendar.merge(
        df_abs_raw,
        on=["startTime", "settlementDate", "settlementPeriod"],
        how="left"
    )

    # Identify empty SPs before filling
    fuel_cols = df_abs.columns.difference(["startTime", "settlementDate", "settlementPeriod"])
    empty_mask = df_abs[fuel_cols].isna().all(axis=1)
    empty_count = empty_mask.sum()
    empty_times = df_abs.loc[empty_mask, "startTime"]

    print(f"\n[INFO] {name_prefix}: {empty_count} SPs had no BOALF actions (filled with 0).")
    if empty_count > 0:
        print(f"StartTimes with no actions for {name_prefix}:")
        print(empty_times.to_list())  # Or just print head() if too many

    # Fill NaNs with 0
    df_abs = df_abs.fillna(0)

    # Set index and normalise settlementDate
    df_abs.set_index("startTime", inplace=True)
    df_abs["settlementDate"] = df_abs["settlementDate"].dt.normalize()

    # Ensure consistent float format
    float_cols = df_abs.select_dtypes(include=['float64']).columns
    df_abs[float_cols] = df_abs[float_cols].round(3).astype('float32')
    df_abs.sort_index(inplace=True)

    # Save ABS file
    df_abs.to_csv(FINAL_OUTPUT_DIR / f"{name_prefix}_ABS.csv")

    # # Create percentage version
    # fuel_cols = [c for c in df_abs.columns if c not in ["settlementDate", "settlementPeriod"]]
    # total_mwh = df_abs[fuel_cols].sum(axis=1)
    # df_pct = df_abs.copy()
    # for col in fuel_cols:
    #     df_pct[col] = (df_pct[col] / total_mwh).replace([np.inf, -np.inf], np.nan).fillna(0) * 100

    # # Save PCT file
    # df_pct.to_csv(FINAL_OUTPUT_DIR / f"{name_prefix}_PCT.csv")

    return df_abs #, df_pct


# 1 & 2: All actions
make_wide_variants(df_energy, "BIDOFFER_ALL", calendar)

# 3 & 4: Non-flagged
df_energy_nonflagged = df_energy[(~df_energy["soFlag"]) & (~df_energy["storFlag"])].copy()
make_wide_variants(df_energy_nonflagged, "BIDOFFER_NONFLAGGED", calendar)



[INFO] BIDOFFER_ALL: 174 SPs had no BOALF actions (filled with 0).
StartTimes with no actions for BIDOFFER_ALL:
[Timestamp('2021-06-28 23:00:00'), Timestamp('2021-06-28 23:30:00'), Timestamp('2021-06-29 00:00:00'), Timestamp('2021-06-29 00:30:00'), Timestamp('2021-06-29 01:00:00'), Timestamp('2021-06-29 01:30:00'), Timestamp('2021-06-29 02:00:00'), Timestamp('2021-06-29 02:30:00'), Timestamp('2021-06-29 03:00:00'), Timestamp('2021-06-29 03:30:00'), Timestamp('2021-06-29 04:00:00'), Timestamp('2021-06-29 04:30:00'), Timestamp('2021-06-29 05:00:00'), Timestamp('2021-06-29 05:30:00'), Timestamp('2021-06-29 06:00:00'), Timestamp('2021-06-29 06:30:00'), Timestamp('2021-06-29 07:00:00'), Timestamp('2021-06-29 07:30:00'), Timestamp('2021-06-29 08:00:00'), Timestamp('2021-06-29 08:30:00'), Timestamp('2021-06-29 09:00:00'), Timestamp('2021-06-29 09:30:00'), Timestamp('2021-06-29 10:00:00'), Timestamp('2021-06-29 10:30:00'), Timestamp('2021-06-29 11:00:00'), Timestamp('2021-06-29 11:30:00'), Ti

,settlementDate,settlementPeriod,GAS,OTHER
startTime,,,,
2021-06-28 23:00:00,2021-06-29,1,0.0,0.0
2021-06-28 23:30:00,2021-06-29,2,0.0,0.0
2021-06-29 00:00:00,2021-06-29,3,0.0,0.0
2021-06-29 00:30:00,2021-06-29,4,0.0,0.0
2021-06-29 01:00:00,2021-06-29,5,0.0,0.0
...,...,...,...,...
2025-07-01 20:30:00,2025-07-01,44,0.0,0.0
2025-07-01 21:00:00,2025-07-01,45,0.0,0.0
2025-07-01 21:30:00,2025-07-01,46,0.0,0.0
